## Set Up

### Dependencies

In [ ]:
# Installing missing dependencies
!pip install omegaconf coloredlogs wandb

### Clean everything

In [ ]:
# Removes the GitHub repository
!rm -rf aml23-ego

In [ ]:
# Removes the Epic Kitchens frames
!rm -rf ek_data

### Clone GitHub repository

In [ ]:
# Clone the GitHub repository
!git clone https://github.com/RONDEL-Ewen/aml23-ego.git aml23-ego

### Mount Google Drive

In [ ]:
# Mount the Google Drive
from google.colab import drive
drive.mount('/content/drive')

### Unzip Epic Kitchens frames

In [ ]:
%%bash

# As read and write operations from google drive are slow, we suggest to copy and unzip
# the dataset in a local directory on the Colab's machine.
mkdir -p ek_data/frames

# Copy the *.zip files of Epic-Kitchens
cp /content/drive/MyDrive/EPIC-Kitchens-P08/*.zip ./ek_data

# Unzip and remove the original .zip files
for file in ./ek_data/*.zip; do
  fn=$(basename $file)
  fn=${fn/.zip/}
  ls -lah $file
  mkdir -p ek_data/frames/$fn
  unzip $file -d ek_data/frames/$fn
  rm $file  # Remove the original .zip file after extraction
done

## Extract RGB features with I3D model

In [ ]:
%%bash

# Replace the following path with the path of your codebase
cd aml23-ego

python save_feat.py name=I3D_features_25_dense \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames

## RGB Features Clustering

In [ ]:
%%bash

cd aml23-ego

features_path='./saved_features/I3D_features_5_dense_D1_test.pkl'
labels_path='./train_val/D1_test.pkl'

python features_pattern_using_cluster.py "$features_path" "$labels_path"

Cluster 0:
  - mix egg in rice
  - pace curry in pot
  - rinse knife
  - wipe hands
  - close
  - put avocado in plastic bag
  - get mint leaves
  - stirring rice
  - open dishwasher
  - place both plastic bags
  - clean filter
  - wash tomatoes
  - place chopping board over sink
  - place bread
  - pour soy sauce
  - still mix food in pan
  - place cheese
  - put corn in pan
  - place pan on sink
  - get knife
  - take access water out
  - pour water over chopping board
  - place scissors in drawer
  - wash leaves
  - take the bread
  - get meat
  - get bowl
  - get kitchen towel
  - close washing machine
  - place sponge
  - pour cooked rice in pan
  - get milk
  - place plate
  - open cupboard
  - get sponge
  - put tap water
  - place fork
  - place spoon back
  - place sponge back
  - wash hands
  - place lid on pan
  - clean colander
  - clean pan
  - clean counter
  - wash chopping board with sponge
  - clean coffee filter
  - get vegetable rice
  - pour whisked egg
  - wipe off

To see the plot, run directly features_pattern_using_cluster.py in your IDE / own environment (it is not shown on Google Colab).

## RGB Features Classifying

### Multi-Layer Perceptron Classifier

In [ ]:
%%bash

cd aml23-ego

python train_classifier.py name=mlp_training \
  config=configs/default.yaml \
  action=train \
  models_dir=models \
  train.num_iter=5000 \
  train.lr_steps=3000 \
  train.num_frames_per_clip.RGB=25 \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=saved_features \
  dataset.RGB.features_name=I3D_features_25_dense \
  models.RGB.model=MLPClassifier

In [ ]:
%%bash

cd aml23-ego

python train_classifier.py name=mlp_test \
  config=configs/default.yaml \
  action=test \
  models_dir=models \
  test.num_frames_per_clip.RGB=5 \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=saved_features \
  dataset.RGB.features_name=I3D_features_5_dense \
  models.RGB.model=MLPClassifier

### Long Short-Term Memory Classifier

In [ ]:
%%bash

cd aml23-ego

python train_classifier.py name=lstm_test \
  config=configs/default.yaml \
  action=test \
  models_dir=models \
  train.num_frames_per_clip.RGB=5 \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=saved_features \
  dataset.RGB.features_name=I3D_features_5_dense \
  models.RGB.model=LSTMClassifier

In [ ]:
%%bash

cd aml23-ego

python train_classifier.py name=lstm_training \
  config=configs/default.yaml \
  action=train \
  models_dir=models \
  train.num_frames_per_clip.RGB=5 \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=saved_features \
  dataset.RGB.features_name=I3D_features_5_dense \
  models.RGB.model=LSTMClassifier

## Extract right/left EMG data

In [ ]:
%%bash

cd aml23-ego

python extract_emg.py source_filepath=../drive/MyDrive/emg_data/2022-06-14_16-38-43_streamLog_actionNet-wearables_S04.hdf5 \
  left_filepath=./S04/emg_data/S04_left.pkl \
  right_filepath=./S04/emg_data/S04_right.pkl

## Extract frames from videos

In [ ]:
%%bash

cd aml23-ego

# Vidéo 1
python extract_frames.py video_path=../drive/MyDrive/S04/PG1_2022-06-14-16-36-51.avi \
  frames_folder=./S04/frames/PG1/

In [ ]:
%%bash

cd aml23-ego

# Vidéo 2
python extract_frames.py video_path=../drive/MyDrive/S04/PG2_2022-06-14-16-36-51.avi \
  frames_folder=./S04/frames/PG2/

In [ ]:
%%bash

cd aml23-ego

# Vidéo 3
python extract_frames.py video_path=../drive/MyDrive/S04/PG3_2022-06-14-16-36-51.avi \
  frames_folder=./S04/frames/PG3/

In [ ]:
%%bash

cd aml23-ego

# Vidéo 4
python extract_frames.py video_path=../drive/MyDrive/S04/PG4_2022-06-14-16-36-51.avi \
  frames_folder=./S04/frames/PG4/

In [ ]:
%%bash

cd aml23-ego

# Vidéo 5
python extract_frames.py video_path=../drive/MyDrive/S04/PG5_2022-06-14-16-36-51.avi \
  frames_folder=./S04/frames/PG5/

In [ ]:
# To check if everything worked well
import os

frames_folder = '/content/aml23-ego/S04/frames/PG5/'

frames_number = len([name for name in os.listdir(frames_folder) if os.path.isfile(os.path.join(frames_folder, name))])
print(f"Number of frames: {frames_number}")

with open('/content/aml23-ego/S04/frames/PG5/timestamps.txt', 'r') as file:
    lines_number = 0
    for line in file:
        lines_number += 1
print(f"Number of timestamps : {lines_number}")


## Pair frames and timestamps

In [ ]:
%%bash

cd aml23-ego

python pair_frames_timestamps.py frames_dir=./S04/frames \
  video=/PG1 \
  timestamps_file=/PG1/timestamps.txt \
  output_file=PG1_frames_timestamps.pkl

In [ ]:
%%bash

cd aml23-ego

python pair_frames_timestamps.py frames_dir=./S04/frames \
  video=/PG2 \
  timestamps_file=/PG2/timestamps.txt \
  output_file=PG2_frames_timestamps.pkl

In [ ]:
%%bash

cd aml23-ego

python pair_frames_timestamps.py frames_dir=./S04/frames \
  video=/PG3 \
  timestamps_file=/PG3/timestamps.txt \
  output_file=PG3_frames_timestamps.pkl

In [ ]:
%%bash

cd aml23-ego

python pair_frames_timestamps.py frames_dir=./S04/frames \
  video=/PG4 \
  timestamps_file=/PG4/timestamps.txt \
  output_file=PG4_frames_timestamps.pkl

In [ ]:
%%bash

cd aml23-ego

python pair_frames_timestamps.py frames_dir=./S04/frames \
  video=/PG5 \
  timestamps_file=/PG5/timestamps.txt \
  output_file=PG5_frames_timestamps.pkl

## Agreggate left and right arms EMG data & RGB frames

In [ ]:
%%bash

cd aml23-ego

python aggregate_data.py

## Split EMG/RGB data into training & testing dataset

In [ ]:
%%bash

cd aml23-ego

python split_train_test.py

## Recreate EMG data

In [ ]:
%%bash

cd aml23-ego

python train_autoencoder.py train_file=../drive/MyDrive/train_test/train_data.pkl \
  test_file=../drive/MyDrive/train_test/test_data.pkl